In [126]:
import numpy as np
import tensorflow as tf

In [1087]:
tf.reset_default_graph()

In [1088]:
sentence = 'Hi! My name is Seohyun Choi'

In [1089]:
ch_to_idx = {ch : idx for idx, ch in enumerate(list(set(sentence)))}
idx_to_ch = {idx : ch for idx, ch in enumerate(list(set(sentence)))}

In [1090]:
n_nodes = 50
n_characters= len(ch_to_idx)

In [1091]:
whh = tf.Variable(tf.random_normal(shape=[n_nodes, n_nodes], dtype=tf.float32))
wxh = tf.Variable(tf.random_normal(shape=[ n_characters, n_nodes], dtype=tf.float32))
why = tf.Variable(tf.random_normal(shape=[ n_nodes,  n_characters], dtype=tf.float32))
bh = tf.Variable(tf.random_normal(shape=[n_nodes], dtype=tf.float32))
by = tf.Variable(tf.random_normal(shape=[n_characters], dtype=tf.float32))

In [1092]:
train_x = tf.placeholder(shape=[None, n_characters], dtype=tf.float32)
train_y = tf.placeholder(shape=[None ,n_characters], dtype=tf.float32)

### Making each softmax at every hidden state

In [1093]:
opt = tf.train.AdamOptimizer(learning_rate=0.001)
hidden_layer = {}
output ={}
pred={}
loss={}
training={}
for i in range(len(sentence)):
    if i!=0:
        hidden_layer[i] = tf.add(tf.matmul(train_x, wxh), tf.matmul(hidden_layer[i-1], whh))
    else :
        hidden_layer[i] = tf.matmul(train_x, wxh)
    hidden_layer[i]= tf.add(hidden_layer[i], bh)
    hidden_layer[i] = tf.tanh(hidden_layer[i])
    output[i] = tf.add(tf.matmul(hidden_layer[i], why), by)  
    pred[i] = tf.nn.softmax(output[i])
    loss[i] = tf.reduce_mean(tf.reduce_sum(train_y * -tf.log(pred[i])))
    training[i] = opt.minimize(loss[i])

In [1094]:
def one_hot(x, depth):
    result = []
    for num in x:
        t = np.zeros(depth)
        t[num]=1
        result.append(t)
    return(result)

In [1095]:
x = sentence[:-1]
Y = sentence[1:]

In [1096]:
xx =[]
for ch in x:
    idx = ch_to_idx[ch]
    xx.append(idx)
xx = one_hot(xx, depth = n_characters)

yy =[]
for ch in Y:
    idx = ch_to_idx[ch]
    yy.append(idx)
yy = one_hot(yy, depth = n_characters)

In [1097]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for _ in range(20000):
    for i in range(len(sentence)-1):
        sess.run(training[i], feed_dict={train_x : np.reshape(xx[i], newshape=[1,n_characters]), 
                                      train_y : np.reshape(yy[i], newshape=[1,n_characters])})
    if _ % 2000 ==0: print(sum(sess.run(loss,feed_dict={train_x : np.reshape(xx[i], newshape=[1,n_characters]), 
                                                     train_y : np.reshape(yy[i], newshape=[1,n_characters])}).values()))

334.1987777352333
72.48257341980934
46.89181727170944
53.41988143324852
170.44070886130794
108.3908291310072
61.093458987772465
137.52346435002983
60.64919745665975
76.7251137830317


In [1100]:
# model test
test_x = xx
print('H', end='')
for i in range(len(sentence)-1):
    t = sess.run(pred[i], feed_dict={train_x: np.reshape(test_x[i], newshape=[1,n_characters])})
    tt = np.argmax(t)
    result = idx_to_ch[tt]
    print(result, end='')

Hi! My name is Seohyun Choi

### Making one combined softmax for all hidden state

In [200]:
hidden_layer = {}
pred =[]
for i in range(len(sentence)-1):
    if i!=0:
        hidden_layer[i] = tf.add(tf.matmul(tf.reshape(train_x[i], [1,n_characters]), wxh), tf.matmul(hidden_layer[i-1], whh))
    else :
        hidden_layer[i] = tf.matmul(tf.reshape(train_x[i], [1,n_characters]), wxh)
    hidden_layer[i]= tf.add(hidden_layer[i], bh)
    hidden_layer[i] = tf.tanh(hidden_layer[i])
    output = tf.add(tf.matmul(hidden_layer[i], why), by)
    pred.append(tf.nn.softmax(output)) # 밑에처럼하나 똑같네 
#pred = tf.nn.softmax(output)
log_pred = tf.reshape(tf.log(pred), [-1,n_characters])
loss = tf.reduce_mean(tf.reduce_sum(train_y * -log_pred))
opt = tf.train.AdamOptimizer(learning_rate=0.001)
training = opt.minimize(loss)

In [203]:
def one_hot(x, depth):
    result = []
    for num in x:
        t = np.zeros(depth)
        t[num]=1
        result.append(t)
    return(result)

In [204]:
x = sentence[:-1]
Y = sentence[1:]

In [205]:
xx =[]
for ch in x:
    idx = ch_to_idx[ch]
    xx.append(idx)
xx = one_hot(xx, depth = n_characters)

yy =[]
for ch in Y:
    idx = ch_to_idx[ch]
    yy.append(idx)
yy = one_hot(yy, depth = n_characters)

In [206]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for iteration in range(10000):
    sess.run(training, feed_dict={train_x : xx, train_y : yy})
    if iteration % 1000 ==0: print(sess.run(loss, feed_dict={train_x : xx, train_y : yy}))

316.18646
4.4459343
0.48795283
0.1912252
0.09577693
0.053253867
0.031076472
0.01856732
0.011221308
0.0068155187


In [208]:
# model test
test_x = xx
print('H', end='')
preds = sess.run(pred, feed_dict={train_x: test_x})
for t in preds:
    tt = np.argmax(t)
    result = idx_to_ch[tt]
    print(result, end='')

Hi! My name is Seohyun Choi